<a href="https://colab.research.google.com/github/PolinaBoeva/predictive_models_dota2/blob/main/experiments/Boeva/%D0%91%D0%B5%D0%B9%D0%B7%D0%BB%D0%B0%D0%B9%D0%BD_%D0%B8_%D0%BB%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D1%8B%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D0%91%D0%BE%D0%B5%D0%B2%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [34]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
df_team_cleaned = pd.read_csv('df_team_cleaned.csv')
df_team_cleaned = df_team_cleaned.drop(df_team_cleaned.columns[0], axis=1)
df_team_cleaned.head()

,match_id,previous_kills_avr_team_1_mean,previous_kills_avr_team_1_max,previous_kills_avr_team_1_min,previous_hero_kills_avr_team_1_mean,previous_hero_kills_avr_team_1_max,previous_hero_kills_avr_team_1_min,previous_courier_kills_avr_team_1_mean,previous_courier_kills_avr_team_1_max,previous_courier_kills_avr_team_1_min,...,previous_win_avr_team_2_mean,previous_win_avr_team_2_max,previous_win_avr_team_2_min,previous_duration_avr_team_2_mean,previous_duration_avr_team_2_max,previous_duration_avr_team_2_min,previous_first_blood_time_avr_team_2_mean,previous_first_blood_time_avr_team_2_max,previous_first_blood_time_avr_team_2_min,radiant_win
0,6947109699,6.600000,11.0,2.0,6.600000,11.0,2.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,32.400000,32.400000,32.400000,2.066667,2.066667,2.066667,1
1,6947237459,9.400000,15.0,3.0,9.400000,15.0,3.0,0.600000,1.000000,0.0,...,0.000000,0.0,0.000000,37.366667,37.366667,37.366667,0.866667,0.866667,0.866667,1
2,6947793080,9.800000,24.0,3.0,9.800000,24.0,3.0,0.400000,2.000000,0.0,...,0.000000,0.0,0.000000,38.316667,38.316667,38.316667,1.833333,1.833333,1.833333,1
3,6947885612,7.000000,12.5,2.5,7.000000,12.5,2.5,0.200000,0.500000,0.0,...,1.000000,1.0,1.000000,33.883333,33.883333,33.883333,1.791667,1.791667,1.791667,1
4,6947952638,8.666667,15.0,4.0,8.666667,15.0,4.0,0.133333,0.666667,0.0,...,0.466667,1.0,0.333333,31.711111,31.711111,31.711111,1.933333,1.933333,1.933333,0


In [55]:
df_team_cleaned.shape

(52293, 182)

In [22]:
df_team_cleaned.memory_usage(deep=True).sum() / 1024 / 1024 # Mb

72.61155700683594

In [23]:
fcols = df_team_cleaned.select_dtypes('float').columns
icols = df_team_cleaned.select_dtypes('integer').columns

df_team_cleaned[fcols] = df_team_cleaned[fcols].apply(pd.to_numeric, downcast='float')
df_team_cleaned[icols] = df_team_cleaned[icols].apply(pd.to_numeric, downcast='integer')

In [24]:
df_team_cleaned.memory_usage(deep=True).sum() / 1024 / 1024 # Mb

42.73913288116455

In [25]:
y = df_team_cleaned['radiant_win']
X = df_team_cleaned.drop(columns=['radiant_win', 'match_id'])

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [ ]:
metrics_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"])

In [68]:
def evaluate_model(y_test, predictions, X_test, model, model_name):
    global metrics_df
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    try:
        roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # Для AUC-ROC используем predict_proba
    except ValueError:
        roc_auc = 0.5
    print(f"Metrics for {model_name}:")
    print(f"Accuracy: {round(accuracy, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall: {round(recall, 3)}")
    print(f"F1 Score: {round(f1, 3)}")
    print(f"ROC-AUC: {round(roc_auc, 3)}")

    new_metrics = pd.DataFrame([[model_name, round(accuracy, 3), round(precision, 3),
                                 round(recall, 3), round(f1, 3), round(roc_auc, 3)]],
                                columns=metrics_df.columns)

    metrics_df = pd.concat([metrics_df, new_metrics], ignore_index=True)


In [42]:
# Бейзлайн модель. Увеличила количество интераций для сходимости
pipeline = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression(max_iter=500))
    ])

# Обучение пайплайна
pipeline.fit(X_train, y_train)

# Прогнозирование на тестовой выборке
predictions = pipeline.predict(X_test)

baseline_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'baseline')
baseline_metrics

Metrics for baseline:
Accuracy: 0.592
Precision: 0.596
Recall: 0.604
F1 Score: 0.6
ROC-AUC: 0.638


In [87]:
param_distributions = {
    'model__C': np.logspace(-5, 5, 11),
    'model__penalty': ['l1', 'l2', 'none'],
    'model__solver': ['liblinear', 'saga'],
    'model__max_iter': [100, 500, 1000, 5000]
}

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=5, scoring='accuracy', random_state=42, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

# Выводим лучшие параметры и наилучший результат
print("Best parameters for Logistic Regression:", random_search.best_params_)
print("Best accuracy:", random_search.best_score_)

# Проверьте, какой параметр C был выбран для наилучшей модели
best_C = random_search.best_params_['model__C']
best_penalty = random_search.best_params_['model__penalty']

print(f"Best C: {best_C}, Best penalty: {best_penalty}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Logistic Regression: {'model__solver': 'saga', 'model__penalty': 'l1', 'model__max_iter': 500, 'model__C': 100000.0}
Best accuracy: 0.584793275060284
Best C: 100000.0, Best penalty: l1


In [88]:
best_model = random_search.best_estimator_
predictions = best_model.predict(X_test)
logistic_metrics = evaluate_model(y_test, predictions, X_test, best_model, 'logistic')
logistic_metrics

Metrics for logistic:
Accuracy: 0.592
Precision: 0.596
Recall: 0.604
F1 Score: 0.6
ROC-AUC: 0.638


Подбор гиперпараметров не изменил качество модели. Большое значение C указывает на то, что регуляризация в модели не нужна.




Попробуем обучить модель только на колонках с mean. Использование только средних значений может улучшить модель, если вариативность данных не имеет решающего значения для предсказания, или в том случае, когда данные имеют устойчивые закономерности. Использование только mean упростит сложность обучения из-за уменьшения количества признаков.

In [46]:
df_mean = df_team_cleaned.loc[:, df_team_cleaned.columns.str.contains('mean|radiant_win')]

In [47]:
y = df_mean['radiant_win']
X = df_mean.drop(columns=['radiant_win'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

In [48]:
# Модель на средних
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())])

param_distributions = {
    'model__C': [0.01, 1, 0.1, 100],
    'model__penalty': ['l1', 'l2'],
    'model__solver': ['liblinear', 'saga'],
    'model__max_iter': [100, 500, 5000]
}

random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=3, n_jobs=-1)

random_search.fit(X_train, y_train)

print(f"Лучшие гиперпараметры: {random_search.best_params_}")

Лучшие гиперпараметры: {'model__solver': 'liblinear', 'model__penalty': 'l2', 'model__max_iter': 500, 'model__C': 1}


In [50]:
best_model = random_search.best_estimator_
predictions = best_model.predict(X_test)
logistic_mean_metrics = evaluate_model(y_test, predictions, X_test, best_model, 'logistic_mean')
logistic_mean_metrics

Metrics for logistic_mean:
Accuracy: 0.596
Precision: 0.6
Recall: 0.607
F1 Score: 0.603
ROC-AUC: 0.639


Модель только на средних превосходит модель на всех данных по всем метрикам. Это означает, что модель на средних имеет более высокую точность, полноту и сбалансированную метрику F1, что делает ее более эффективной в задаче предсказания победы команды Radiant. Однако различия в этих метриках невелики, и обе модели имеют схожее качество.

Рассмотрим другие модели





In [53]:
y = df_team_cleaned['radiant_win']
X = df_team_cleaned.drop(columns=['radiant_win', 'match_id'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

In [67]:
# SVM
from sklearn.svm import SVC

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True))
])

pipeline.fit(X_train, y_train)

prediction = pipeline.predict(X_test)
pred_prob = pipeline.predict_proba(X_test)[:, 1]  # Вероятности для положительного класса
svm_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'svm')
svm_metrics

Metrics for svm:
Accuracy: 0.504
Precision: 0.505
Recall: 0.966
F1 Score: 0.664
ROC-AUC: 0.637


Метрики указывают на то, что модель svm на данных склонна к переобучению на положительный класс, что подтверждается высокой полнотой и низкой точностью.


In [73]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Подбор гиперпараметров LDA
param_distributions = {
    'model__solver': ['lsqr', 'eigen', 'svd'],
    'model__priors': [None, 'uniform'],
    'model__n_components': np.arange(1, 4),
}

lda_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearDiscriminantAnalysis())
])

random_search_lda = RandomizedSearchCV(lda_pipeline, param_distributions, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search_lda.fit(X_train, y_train)

# Лучшие параметры
print("Best parameters for LDA:", random_search_lda.best_params_)

Best parameters for LDA: {'model__solver': 'lsqr', 'model__priors': None, 'model__n_components': 1}


In [77]:
best_model = random_search_lda.best_estimator_
predictions = best_model.predict(X_test)
LDA_metrics = evaluate_model(y_test, predictions, X_test, best_model, 'LDA')
LDA_metrics

Metrics for LDA:
Accuracy: 0.59
Precision: 0.594
Recall: 0.602
F1 Score: 0.598
ROC-AUC: 0.638


In [83]:
# Наивный байесовский классификатор
from sklearn.naive_bayes import GaussianNB

nb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GaussianNB())
])

nb_pipeline.fit(X_train, y_train)
nb_predictions = nb_pipeline.predict(X_test)

nb_metrics = evaluate_model(y_test, nb_predictions, X_test, nb_pipeline, 'Naive Bayes')
nb_metrics

Metrics for Naive Bayes:
Accuracy: 0.557
Precision: 0.611
Recall: 0.344
F1 Score: 0.441
ROC-AUC: 0.6


In [86]:
metrics_df

,Model,Accuracy,Precision,Recall,F1 Score,ROC-AUC
0,baseline,0.592,0.596,0.604,0.600,0.638
2,logistic_mean,0.596,0.600,0.607,0.603,0.639
3,svm,0.504,0.505,0.966,0.664,0.637
4,LDA,0.590,0.594,0.602,0.598,0.638
5,Naive Bayes,0.557,0.611,0.344,0.441,0.600


Логистическая регрессия и её модификации, а также LDA, показывают схожие результаты по значениям accuracy, precision, recall и F1 score. Несмотря на высокое значение recall, SVM имеет очень низкую accuracy и precision. Хотя наивный байесовский классификатор показывает хорошую precision, его recall слишком низкий.
Для повышения качества предсказаний, можно попробовать использовать более сложные модели.

In [91]:
# Дерево решений
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeClassifier())
])

pipeline.fit(X_train, y_train)

predictions = pipeline.predict(X_test)

DT_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'Decision Tree')
DT_metrics

Metrics for Decision Tree:
Accuracy: 0.547
Precision: 0.553
Recall: 0.551
F1 Score: 0.552
ROC-AUC: 0.547


Дерево решений в данном случае показывает относительно слабые результаты, модель не очень хорошо отделяет положительный класс от отрицательного, о чем свидетельствует низкий ROC-AUC.

In [92]:
# Случайный лес
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)

predictions = pipeline.predict(X_test)

RT_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'Random Forest')
RT_metrics

Metrics for Random Forest:
Accuracy: 0.597
Precision: 0.601
Recall: 0.606
F1 Score: 0.604
ROC-AUC: 0.625


Random Forest показывает значительное улучшение по сравнению с Decision Tree. Модель достигает лучших результатов по всем метрикам, включая accuracy, precision, recall, F1 score и ROC-AUC.

In [ ]:
pip install catboost

In [ ]:
# catboost
from catboost import CatBoostClassifier

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', CatBoostClassifier())
])

pipeline.fit(X_train, y_train)

In [95]:
predictions = pipeline.predict(X_test)
catboost_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'CatBoost')
catboost_metrics

Metrics for CatBoost:
Accuracy: 0.601
Precision: 0.603
Recall: 0.616
F1 Score: 0.61
ROC-AUC: 0.655


CatBoost показывает достаточно хорошие результаты по большинству метрик, особенно по recall и F1 Score, что говорит о том, что модель эффективно находит положительные примеры.

In [100]:
metrics_df

,Model,Accuracy,Precision,Recall,F1 Score,ROC-AUC
0,baseline,0.592,0.596,0.604,0.600,0.638
1,logistic_mean,0.596,0.600,0.607,0.603,0.639
2,svm,0.504,0.505,0.966,0.664,0.637
3,LDA,0.590,0.594,0.602,0.598,0.638
4,Naive Bayes,0.557,0.611,0.344,0.441,0.600
5,logistic,0.592,0.596,0.604,0.600,0.638
6,Decision Tree,0.547,0.553,0.551,0.552,0.547
7,Random Forest,0.597,0.601,0.606,0.604,0.625
8,CatBoost,0.601,0.603,0.616,0.610,0.655


Лучшие модели по точности и сбалансированным метрикам: CatBoost и Random Forest. Модель CatBoost демонстрирует лучшие результаты по точности (0.601) и ROC-AUC (0.655), что делает её одной из самых стабильных моделей.
ROC-AUC (0.638) baseline модели указывает на то, что она лучше, чем модели, такие как Decision Tree и Naive Bayes, но хуже, чем более сложные алгоритмы, такие как CatBoost и Random Forest.
Для дальнейшего улучшения возможно:

• Подбор гиперпараметров моделей CatBoost и Random Forest.

• Использование других моделей.

• Future engineering (добавление информации по героям и тд.)
